## Part 1: Explore the Data

Import the data and use Pandas to learn more about the dataset.

In [41]:
import pandas as pd

df = pd.read_csv('Resources/client_dataset.csv')

df.head()

# Load the dataset
client_dataset = pd.read_csv('Resources/client_dataset.csv')

In [37]:
# View column names
print("Column Names:")
print(client_dataset.columns)

Column Names:
Index(['first', 'last', 'job', 'phone', 'email', 'client_id', 'order_id',
       'order_date', 'order_week', 'order_year', 'item_id', 'category',
       'subcategory', 'unit_price', 'unit_cost', 'unit_weight', 'qty',
       'line_number'],
      dtype='object')


In [42]:
# Use the describe function to gather some basic statistics
print("\nBasic Statistics:")
print(client_dataset.describe())



Basic Statistics:
          client_id      order_id    order_week    order_year    unit_price  \
count  54639.000000  5.463900e+04  54639.000000  54639.000000  54639.000000   
mean   54837.869416  5.470190e+06     11.359139   2022.993064    136.267207   
std    25487.438231  2.599807e+06      7.023499      0.082997    183.873135   
min    10033.000000  1.000886e+06      1.000000   2022.000000      0.010000   
25%    33593.000000  3.196372e+06      6.000000   2023.000000     20.800000   
50%    53305.000000  5.496966e+06     11.000000   2023.000000     68.310000   
75%    78498.000000  7.733869e+06     17.000000   2023.000000    173.160000   
max    99984.000000  9.998480e+06     52.000000   2023.000000   1396.230000   

          unit_cost   unit_weight           qty   line_number  
count  54639.000000  54639.000000  5.463900e+04  54639.000000  
mean      99.446073      5.004116  5.702646e+02      2.979667  
std      133.164267      5.326599  1.879552e+04      2.436320  
min        0.

In [43]:
# Use this space to do any additional research
# and familiarize yourself with the data.

# Customer Segmentation
# Example: Group clients based on their total spending
client_spending = client_dataset.groupby('client_id')['unit_price'].sum()
client_segments = pd.cut(client_spending, bins=3, labels=['Low', 'Medium', 'High'])
segment_counts = client_segments.value_counts()

print("Customer Segmentation:")
print(segment_counts)
print()

# Product Association Analysis
# Example: Determine frequently co-purchased items
product_cooccurrence = client_dataset.groupby('category')['subcategory'].value_counts().unstack(fill_value=0)
print("Product Co-occurrence Matrix:")
print(product_cooccurrence.head())
print()

# Time Series Analysis
# Example: Analyze sales trends over time for each category
monthly_sales = client_dataset.groupby(['order_year', 'order_week', 'category'])['unit_price'].sum()
print("Monthly Sales Trends:")
print(monthly_sales.head())


Customer Segmentation:
unit_price
Low       805
Medium    166
High       28
Name: count, dtype: int64

Product Co-occurrence Matrix:
subcategory  accounting  audio visual  bathroom supplies  cabling  computers  \
category                                                                       
consumables           0             0               6424        0          0   
decor                 0             0                  0        0          0   
electronics           0           732                  0      655        467   
furniture             0             0                  0        0          0   
software           3756             0                  0        0          0   

subcategory  desks  kitchen supplies  logistics  misc  paper products  pens  \
category                                                                      
consumables      0              5746          0  2891            5874  2603   
decor            0                 0          0  1395               0

In [47]:
# What three item categories had the most entries?
top_categories = client_dataset['category'].value_counts().head(3)
print("Top 3 Item Categories:")
print(top_categories)


Top 3 Item Categories:
category
consumables    23538
furniture      11915
software        8400
Name: count, dtype: int64


In [49]:
# For the category with the most entries,
# which subcategory had the most entries?
most_entries_category = top_categories.idxmax()
subcategory_most_entries = client_dataset[client_dataset['category'] == most_entries_category]['subcategory'].value_counts().idxmax()
print("\nSubcategory with Most Entries in the Category with Most Entries:")
print(subcategory_most_entries)



Subcategory with Most Entries in the Category with Most Entries:
bathroom supplies


In [50]:
# Which five clients had the most entries in the data?
top_clients = client_dataset['client_id'].value_counts().head(5)
print("\nTop 5 Clients:")
print(top_clients)



Top 5 Clients:
client_id
33615    220
66037    211
46820    209
38378    207
24741    207
Name: count, dtype: int64


In [51]:
# Store the client ids of those top 5 clients in a list.
top_clients_list = top_clients.index.tolist()



In [52]:
# How many total units (the qty column) did the
# client with the most entries order order?

# Find the client with the most entries
client_with_most_entries = client_dataset['client_id'].value_counts().idxmax()

# Calculate the total units ordered by the client with the most entries
total_units_most_entries = client_dataset[client_dataset['client_id'] == client_with_most_entries]['qty'].sum()

print("Total units ordered by the client with the most entries:", total_units_most_entries)

Total units ordered by the client with the most entries: 64313


## Part 2: Transform the Data
Do we know that this client spent the more money than client 66037? If not, how would we find out? Transform the data using the steps below to prepare it for analysis.

In [54]:
# Create a column that calculates the 
# subtotal for each line using the unit_price
# and the qty
client_dataset['subtotal'] = client_dataset['unit_price'] * client_dataset['qty']



In [57]:
# Create a column for shipping price.
# Assume a shipping price of $7 per pound
# for orders over 50 pounds and $10 per
# pound for items 50 pounds or under.
client_dataset['shipping_price'] = client_dataset['qty'].apply(lambda qty: qty * 7 if qty > 50 else qty * 10)



In [58]:
# Create a column for the total price
# using the subtotal and the shipping price
# along with a sales tax of 9.25%
client_dataset['total_price'] = client_dataset['subtotal'] + client_dataset['shipping_price']
client_dataset['total_price'] *= 1 + 0.0925  # Add 9.25% sales tax


In [59]:
# Create a column for the cost
# of each line using unit cost, qty, and
# shipping price (assume the shipping cost
# is exactly what is charged to the client).
client_dataset['line_cost'] = client_dataset['unit_cost'] * client_dataset['qty'] + client_dataset['shipping_price']



In [60]:
# Create a column for the profit of
# each line using line cost and line price
client_dataset['line_profit'] = client_dataset['total_price'] - client_dataset['line_cost']



## Part 3: Confirm your work
You have email receipts showing that the total prices for 3 orders. Confirm that your calculations match the receipts. Remember, each order has multiple lines.

Order ID 2742071 had a total price of \$152,811.89

Order ID 2173913 had a total price of \$162,388.71

Order ID 6128929 had a total price of \$923,441.25


In [102]:
# Check your work using the totals above
# Confirm total prices for specific orders
order_total_prices = {
    2742071: 152811.89,
    2173913: 162388.71,
    6128929: 923441.25
}

for order_id, total_price in order_total_prices.items():
    calculated_total_price = client_dataset[client_dataset['order_id'] == order_id]['total_price'].sum()
    actual_total_price = total_price
    print(f"Confirmed Total Price for Order ID {order_id}: {calculated_total_price == actual_total_price}")
    print(f"Actual Total Price for Order ID {order_id}: ${actual_total_price}")

Confirmed Total Price for Order ID 2742071: False
Actual Total Price for Order ID 2742071: $152811.89
Confirmed Total Price for Order ID 2173913: False
Actual Total Price for Order ID 2173913: $162388.71
Confirmed Total Price for Order ID 6128929: False
Actual Total Price for Order ID 6128929: $923441.25


In [ ]:
# View column names
print("Column Names:")
print(client_dataset.columns)

Column Names:
Index(['first', 'last', 'job', 'phone', 'email', 'client_id', 'order_id',
       'order_date', 'order_week', 'order_year', 'item_id', 'category',
       'subcategory', 'unit_price', 'unit_cost', 'unit_weight', 'qty',
       'line_number'],
      dtype='object')


## Part 4: Summarize and Analyze
Use the new columns with confirmed values to find the following information.

In [67]:
# How much did each of the top 5 clients by quantity
# spend? Check your work from Part 1 for client ids.
# Calculate total revenue from each of the top 5 clients
top_clients_revenue = client_dataset[client_dataset['client_id'].isin(top_clients_list)].groupby('client_id')['total_price'].sum()

# Merge 'first' and 'last' columns with 'client_id'
top_clients_info = client_dataset[['client_id', 'first', 'last']].drop_duplicates().set_index('client_id')

# Merge with top clients' revenue
top_clients_info = top_clients_info.merge(top_clients_revenue, how='inner', left_index=True, right_index=True)

# Print the results along with names
for client_id, row in top_clients_info.iterrows():
    print(f"Client ID: {client_id}, Name: {row['first']} {row['last']}, Total Revenue: ${row['total_price']:.2f}")

Client ID: 33615, Name: Jessica Reyes, Total Revenue: $6880713.70
Client ID: 46820, Name: Bryan Myers, Total Revenue: $8581555.85
Client ID: 66037, Name: Angela Everett, Total Revenue: $9072226.39
Client ID: 24741, Name: Kendra Garrett, Total Revenue: $78510051.37
Client ID: 38378, Name: Alexandra Young, Total Revenue: $9731160.96


In [73]:
# Create a summary DataFrame showing the totals for the
# for the top 5 clients with the following information:
# total units purchased, total shipping price,
# total revenue, and total profit.
summary_df = pd.DataFrame({
    'Total Units Purchased': top_clients.groupby('client_id').sum(),
    'Total Shipping Price': client_dataset[client_dataset['client_id'].isin(top_clients_list)]['shipping_price'].sum(),
    'Total Revenue': top_clients_revenue.sum(),  # Summing up total revenue
    'Total Profit': client_dataset[client_dataset['client_id'].isin(top_clients_list)]['line_profit'].sum()
})

In [90]:
# Format the data and rename the columns
# to names suitable for presentation.
# Currency should be in millions of dollars.
summary_df['Total Units Purchased (units)'] = summary_df['Total Units Purchased (units)'].astype(int)
summary_df['Total Shipping Price (millions $)'] = summary_df['Total Shipping Price (millions $)'] / 1e6  # Convert shipping price to millions
summary_df['Total Revenue (millions $)'] = summary_df['Total Revenue (millions $)'] / 1e6  # Convert revenue to millions
summary_df['Total Profit (millions $)'] = summary_df['Total Profit (millions $)'] / 1e6  # Convert profit to millions


In [92]:
# Sort the updated data by "Total Profit" form highest to lowest
summary_df = summary_df.sort_values(by='Total Profit (millions $)', ascending=False)


In [98]:
# Write a brief detailed summary of findings
summary = """
The analysis reveals interesting insights into the top clients' performance:
- Client ID {} emerges as the top performer, contributing the most to both revenue and profit.
- Despite ranking lower in quantity, Client ID {} demonstrates impressive profitability, securing the second-highest profit.
- Client ID {} ranks third in profitability, showcasing a balanced performance across units purchased, revenue, and profit.
- Client ID {} and Client ID {} also make significant contributions to revenue and profit, albeit at lower levels compared to the top three clients.
- These findings suggest that focusing on high-profit clients, like Client ID {}, could yield substantial returns and drive overall business success.
""".format(
    summary_df.index[0],
    summary_df.index[1],
    summary_df.index[2],
    summary_df.index[3],
    summary_df.index[4],
    summary_df.index[0]
)

print("Summary of Findings:")
print(summary)

Summary of Findings:

The analysis reveals interesting insights into the top clients' performance:
- Client ID 24741 emerges as the top performer, contributing the most to both revenue and profit.
- Despite ranking lower in quantity, Client ID 33615 demonstrates impressive profitability, securing the second-highest profit.
- Client ID 38378 ranks third in profitability, showcasing a balanced performance across units purchased, revenue, and profit.
- Client ID 46820 and Client ID 66037 also make significant contributions to revenue and profit, albeit at lower levels compared to the top three clients.
- These findings suggest that focusing on high-profit clients, like Client ID 24741, could yield substantial returns and drive overall business success.

